In [1]:
import itertools
import time
import pandas as pd
from joblib import Parallel, delayed

from thefittest.optimizers import GeneticAlgorithm
from thefittest.utils.transformations import GrayCode
from thefittest.benchmarks import Rastrigin

In [2]:
import itertools
import time
import pandas as pd
from joblib import Parallel, delayed

from thefittest.optimizers import GeneticAlgorithm
from thefittest.utils.transformations import GrayCode
from thefittest.benchmarks import Rastrigin


# === Настройки ===
n_runs = 50
n_dimension = 10
left_border, right_border = -5., 5.
population_size = 200
generations = 300

# === Кодирование ===
genotype_to_phenotype = GrayCode()
genotype_to_phenotype.fit(left_border, right_border, n_dimension, h_per_variable=0.001)
num_bits = genotype_to_phenotype.get_bits_per_variable().sum()

fitness_func = Rastrigin()

# === Гиперпараметры ГА ===
selections = ["tournament_3", "tournament_5", "rank", "proportional"]
crossovers = ["one_point", "two_point", "uniform_2", "uniform_prop_2"]
mutations = ["weak", "average", "strong"]
elitism_opts = [True, False]

param_grid = list(itertools.product(selections, crossovers, mutations, elitism_opts))
print(f"Всего комбинаций: {len(param_grid)}")

# === Функция запуска одного эксперимента ===
def run_experiment(selection, crossover, mutation, elitism, run_id):
    optimizer = GeneticAlgorithm(
        fitness_function=fitness_func,
        genotype_to_phenotype=genotype_to_phenotype.transform,
        iters=generations,
        pop_size=population_size,
        str_len=num_bits,
        selection=selection,
        crossover=crossover,
        mutation=mutation,
        elitism=elitism,
        minimization=True,
    )

    t0 = time.time()
    optimizer.fit()
    elapsed = time.time() - t0

    fittest = optimizer.get_fittest()
    return {
        "selection": selection,
        "crossover": crossover,
        "mutation": mutation,
        "elitism": elitism,
        "run_id": run_id,
        "best_fitness": fittest["fitness"],
        "best_point": fittest["phenotype"].tolist(),
        "time_sec": elapsed,
    }

# === Запуск всего перебора ===
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(run_experiment)(sel, cross, mut, elit, run_id)
    for (sel, cross, mut, elit) in param_grid
    for run_id in range(n_runs)
)

# === Сохранение в CSV ===
df = pd.DataFrame(results)
df.to_csv("ga_benchmark_results.csv", index=False)
df.head()


Всего комбинаций: 96


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  

,selection,crossover,mutation,elitism,run_id,best_fitness,best_point,time_sec
0,tournament_3,one_point,weak,True,0,-4.974966,"[-0.0003051944088383607, 0.0003051944088383607...",0.647601
1,tournament_3,one_point,weak,True,1,-7.959836,"[0.9952389672221207, 0.9952389672221207, 0.995...",0.875094
2,tournament_3,one_point,weak,True,2,-7.959983,"[0.9952389672221207, -0.0003051944088383607, -...",0.728668
3,tournament_3,one_point,weak,True,3,-3.980009,"[-0.9952389672221207, -0.9952389672221207, 0.0...",0.931858
4,tournament_3,one_point,weak,True,4,-3.980009,"[0.9952389672221207, -0.0003051944088383607, 0...",0.671389
